# Search_pdf_on_GALLICA_from_a_list
* En entrée : liste de titres de documents à rechercher sur Gallica
* En sortie : un fichier avec la liste de tous les titres trouvés et leurs url + un fichier avec la même liste et un comparateur de similarité pour confronter les résultats avec les titres requêtés.

In [1]:
#Import des librairies nécessaires
import os
import re
import urllib
import lxml.etree as etree
import glob
import jellyfish

## Lancer l'API GALLICA et pré-filtrer les résultats
* Format de fichier en entrée : .txt ou .tsv 

* une ligne = identification_du_document (tabulation) titre_à_chercher

In [2]:
def load_clean_tronc(path, erase_ponctuation = True, nb_mots = 10):
#Pour charger les titres à requêter, supprimer la ponctuation et les couper car les titres longs 
#sont plus susceptibles de générer de mauvais résultats dans l'api.
#Path = chemin vers le fichier où sont les titres
#nb_mots = nombre de mots à garder pour les titres-requêtes
#erase_ponctuation = True pour supprimer la ponctuation"""
    with open(path, "r", encoding='utf8') as f :
        lignes=f.readlines()
    if erase_ponctuation == True :
        lignes_clean = [re.sub(" \([\w]+[\'|\’]?\)", "", i) for i in lignes]
        lignes_clean = [re.sub('[^\w\t ]+', ' ', i) for i in lignes_clean]
        lignes_clean = [re.sub('  ', ' ', i) for i in lignes_clean]
        lignes_clean = [re.sub("\n", '', i) for i in lignes_clean]
    else :
        lignes_clean=lignes
    liste_tronquee=[]
    for l in lignes_clean:
        try:
            idMoreau, titre=re.split("\t",l)
        except:
            print("erreur à la ligne suivante : "+l)
        if len(titre)<40:
            liste_tronquee.append(idMoreau+"\t"+titre)
        if len(titre)>40:
            try:
                titre_coupe=re.split(" ",titre)
            except:
                print("erreur dans le titre suivant : "+titre)
            titre_tronque=titre_coupe[0:nb_mots]
            titre_tronque_str = '%20'.join(titre_tronque)
            liste_tronquee.append(idMoreau+"\t"+titre_tronque_str)
    return liste_tronquee

In [3]:
def launch_api_gallica(path, max_results = 40):
#Cette fonction transforme les titres en requête URL valide dans l'api gallica, puis enregistre les résultats de
#l'api dans un fichier XML (identifiant_du_titre.xml) 
#paramètres API gallica : cherche toutes les monographies contenant les mots du titre, publiées au XVIIe siècle
    liste_tronquee = load_clean_tronc(path, erase_ponctuation = True, nb_mots = 10)
    try:
        os.mkdir('sortie_xml')
    except:
        pass
    chaine1 = "https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20gallica%20any%20%22"
    chaine2 = "%22%20and%20dc.type%20any%20monographie%20and%20(century%20adj%20%2217%22)&maximumRecords="
    chaine3 = "%s" %max_results
    list_no_results=[]
    for l in liste_tronquee:
        idMoreau, titre=re.split("\t",l)
        titre_ascii=urllib.parse.quote(titre) #transformer les titres au format ascii
        url= chaine1 + titre_ascii + chaine2 + chaine3
        try:
            txt = urllib.request.urlretrieve(url, 'sortie_xml/%s.xml' %idMoreau)
        except:
            list_no_results.append(l)
    if not(list_no_results):
        print("Tous les titres ont été requêtés et les résultats enregistrés dans des XML.")
    else:
        #relancer l'api sur les titres qui n'ont pas pu être requêtés avec une autre clé GBOOKS:
        print("Nombre de titres pour lesquels la recherche API n'a pas fonctionné : "+str(len(list_no_results))+". Liste des titres enregistrée au format .txt au même niveau que ce script.")

In [4]:
def download_filtered_url(nombre = 40, regex="1648|1649|1650|1651|1652|1653|1654"):
#enregistre dans une liste les résultats de l'api correspondants à la recherche avec : 
#date de publi + titre du document + adresse URL où télécharger le doc
#Enregistre les résultats au format .txt
#nombre = nombre max de résultats de la requête de l'api selectionné
#regex = mettre la date ou les dates de publication qui serviront à filtrer les résultats
    p = re.compile("%s" %regex,re.IGNORECASE)
    liste_url=[]
    for filepath in glob.iglob('sortie_xml/*.xml'):
            if filepath.endswith('.xml'):
                tree = etree.parse("%s" %filepath)
                for i in range(nombre): 
                    for date in tree.xpath("/srw:searchRetrieveResponse/srw:records[1]/srw:record[%s]/srw:recordData[1]/oai_dc:dc[1]/dc:date[1]" %str(i+1),
                              namespaces={'srw': 'http://www.loc.gov/zing/srw/','oai_dc':"http://www.openarchives.org/OAI/2.0/oai_dc/",'dc':'http://purl.org/dc/elements/1.1/'}):
                        m = p.search(date.text)
                        if m:
                            v_date=m.group()
                            for title in tree.xpath("/srw:searchRetrieveResponse/srw:records[1]/srw:record[%s]/srw:recordData[1]/oai_dc:dc[1]/dc:title[1]" %str(i+1),
                                       namespaces={'srw': 'http://www.loc.gov/zing/srw/','oai_dc':"http://www.openarchives.org/OAI/2.0/oai_dc/",'dc':'http://purl.org/dc/elements/1.1/'}):
                                v_titre=title.text
                            for link in tree.xpath("/srw:searchRetrieveResponse/srw:records[1]/srw:record[%s]/srw:extraRecordData[1]/link[1]" %str(i+1),
                                      namespaces={'srw': 'http://www.loc.gov/zing/srw/'}):
                                v_url=link.text
                            ligne=str(v_date)+"\t"+v_titre+"\t"+v_url
                            liste_url.append(ligne)
    liste_url=list(set(liste_url))
    print("Nombre de documents correspondants aux dates : " + str(len(liste_url)))
    try:
        os.mkdir('output')
    except:
        pass
    open("output/liste_url.txt", "w", encoding="utf8")
    for l in liste_url :
        with open ("output/liste_url.txt", "a", encoding="utf8") as f:
            f.write(l+"\n"+"\n")
    return liste_url

## Passer le résultat de l'API dans le get_similar_titles

In [5]:
def get_tsv(path):
    f = open(path,encoding="utf-8")
    liste = f.readlines()
    f.close()
    return liste

def get_closest(path,title, number = 5):
#En entrée, une chaîne de caractère (et en option le nombre de titres proches à retourner, par défaut : 5)
#En sortie, la liste triée par ordre décroissant de similarité sous forme d'un triplet:
#distance, ID_moreau, titre concerné.
    liste = get_tsv(path)
    mots_titre = set(title.split())
    filtered_list = []
    for l in  liste:
        ID, chaine=re.split("\t",l)
        mots_candidat = set(chaine.split())
        inter = mots_titre.intersection(mots_candidat)
        ##Si moins de 5 mots en commun (ou de la moitié) inutile de garder le candidat
        if len(inter)>5 or len(inter)>(len(mots_titre)/2):
            filtered_list.append([len(inter), ID, chaine])
    f2 = []
    for len_inter, ID, chaine in filtered_list:
        sim = jellyfish.jaro_winkler(title,chaine)
        ## On garde quand la similarité est supérieure à 0.5
        if sim>0.5:
            f2.append([sim, ID, chaine])
    return sorted(f2, reverse=True)[:number]

In [6]:
def download_results(path,nombre = 40, regex="1648|1649|1650|1651|1652|1653|1654"):
#nombre = nombre de sortie api gallica à intégrer dans les résultats enregistrés pour une requête
#enregistrer les résultats dans un fichier .txt à ouvrir avec notepad++ pour plus de lisibilité
#regex=mettre une ou des dates de publication (séparées par des "|") pour filtrer les résultats
    liste_titres = download_filtered_url(nombre, regex)
    dic= {}
    cpt = 0
    open ("output/output_similar.txt", "w", encoding="utf8")
    for lt in liste_titres: #A titre d'exemple
        date, titre,url=re.split("\t",lt)
        with open ("output/output_similar.txt", "a", encoding="utf8") as f:
            f.write("\n"+"-"*30+"\n"+ date + "\t"+ titre + "\n"+ url+"\n")
        print("-"*30)
        print(date, titre, url)
        closest_titles = get_closest(path,titre)
        for dist, ID_moreau, chaine in closest_titles:
            result=f"  D={round(dist, 4)} (ID={ID_moreau}): {chaine[:70]}"
            print(result)
            with open ("output/output_similar.txt", "a", encoding="utf8") as f:
                f.write (result+"\n")

# Exemple d'utilisation :
* Remplacer "essai_titre.txt" par le nom de votre fichier avec les titres à chercher en respectant le format (id_titre+tabulation+titre+saut de ligne). Veiller à ce qu'il se trouve bien au même niveau que ce script.
* Cliquer sur Cell > Run all. Attendre quelques minutes.
* L'ensemble des résultats de l'application se trouvent dans le dossier de sortie "output", créé au même niveau que le script.
* Conseil : ouvrir le fichier "output_similar.txt" avec le logiciel notepad++ pour une meilleure lisibilité.

In [7]:
launch_api_gallica("Liste_Titres.txt",max_results = 40)

Tous les titres ont été requêtés et les résultats enregistrés dans des XML.


In [8]:
download_results("Liste_Titres.txt", nombre = 40, regex="1648|1649|1650|1651|1652|1653|1654")

Nombre de fichiers correspondants trouvés : 120
------------------------------
1649 L'Agatonphile de la France https://gallica.bnf.fr/ark:/12148/bpt6k55771405
------------------------------
1652 Triolets sur la jonction des princes, pour la deroute des Mazarins https://gallica.bnf.fr/ark:/12148/bpt6k54293789
------------------------------
1649 Divine revelation arrivée à un bon religieux, Du retour du Roy, & de la Paix. https://gallica.bnf.fr/ark:/12148/bpt6k857024z
------------------------------
1652 Inventaire des pièces que met et baille par devers vous, nos seigneurs de parlement, la Sagesse éternelle... est demanderesse, en restitution de la monarchie française... https://gallica.bnf.fr/ark:/12148/btv1b8606955g
------------------------------
1650 Dialogue des amours d'un sergent avec une villageoise https://gallica.bnf.fr/ark:/12148/bpt6k53993713
------------------------------
1648 Mémoires pour l'histoire de Navarre et de Flandre, contenant le droit du roi au royaume de Navarre e

------------------------------
1649 Triolets sur Cambray https://gallica.bnf.fr/ark:/12148/bpt6k54293648
------------------------------
1652 L'agreable conferance de deux Normans, s'estans rencontrez sur le Pont-Neuf de cette ville de Paris, traitans sur les affaires du temps present . Dont l'un se nomme Perrin, & l'autre Colas. Dialogue. https://gallica.bnf.fr/ark:/12148/bpt6k1335801
------------------------------
1651 Lettre escrite de Bazas par un Ecclésiastique a un Prestre de Sainct Sulpice,... contenante l'Apostasie d'un Ianséniste nommé Labadie, lequel a renoncé à l'Eglise Romaine. Et a protesté en se faisant qu'il n'a pas changé la croyance du Iansénisme en professant le Calvinisme,... https://gallica.bnf.fr/ark:/12148/bpt6k855244q
------------------------------
1649 Paraphrase sur le bref de sa saincteté envoyé à la Reyne régente mère du Roy, touchant sa reconciliation avec plusieurs des plus signalez de son royaume et le soulagement de son peuple. En vers burlesques https://g

**L'ensemble des résultats de l'application se trouvent dans le dossier de sortie "output".**